In [1]:
import carla
import random
import numpy as np
import time
import cv2

In [ ]:


im_width = 640
im_height = 480

latest_frame = None  

def process_image(image):
    global latest_frame
    i = np.array(image.raw_data)
    i2 = i.reshape((im_height, im_width, 4))
    i3 = i2[:, :, :3]
    latest_frame = i3 
    return i3 / 255.0

actor_list = []

try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(2.0)
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()

    vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
    spawn_point = world.get_map().get_spawn_points()[0]

    ego_vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    ego_vehicle.set_autopilot(False)
    ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
    actor_list.append(ego_vehicle)

    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', f"{im_width}")
    camera_bp.set_attribute('image_size_y', f"{im_height}")
    camera_bp.set_attribute('fov', '110')

    camera_position = carla.Transform(carla.Location(x=2.5, y=0, z=1.5))
    camera = world.spawn_actor(camera_bp, camera_position, attach_to=ego_vehicle)
    actor_list.append(camera)

    camera.listen(lambda image: process_image(image))

    # Main display loop in main thread
    while True:
        if latest_frame is not None:
            cv2.imshow('Camera', latest_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    for actor in actor_list:
        actor.destroy()
    cv2.destroyAllWindows()
    print('ALL CLEANED UP')


In [ ]:


im_width = 640
im_height = 480

latest_frame = None  

def process_image(image):
    global latest_frame
    i = np.array(image.raw_data)
    i2 = i.reshape((im_height, im_width, 4))
    i3 = i2[:, :, :3]  
    latest_frame = i3  
    return i3 / 255.0

actor_list = []


client = carla.Client('localhost', 2000)
client.set_timeout(2.0)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_point = world.get_map().get_spawn_points()[0]

ego_vehicle = world.spawn_actor(vehicle_bp, spawn_point)
ego_vehicle.set_autopilot(False)
ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
actor_list.append(ego_vehicle)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', f"{im_width}")
camera_bp.set_attribute('image_size_y', f"{im_height}")
camera_bp.set_attribute('fov', '110')

camera_position = carla.Transform(carla.Location(x=2.5, y=0, z=1.5))
camera = world.spawn_actor(camera_bp, camera_position, attach_to=ego_vehicle)
actor_list.append(camera)

camera.listen(lambda image: process_image(image))

# Main display loop in main thread
while True:
    if latest_frame is not None:
        cv2.imshow('Camera', latest_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()
print('ALL CLEANED UP')


In [ ]:
#check all the alive actors in the simulation
all_actors= world.get_actors()
for actor in all_actors:
    print(f' ID: {actor.type_id}')

In [ ]:
# destroy all the actors
for e in all_actors:
    if e.type_id.startswith('vehicle.') or e.type_id.startswith('sensor.'):
          e.destroy()
    